In [1]:
# General imports 
import cv2
import numpy as np

In [2]:
# Define directories 

vid_or_dir = '/home/ulascombes/disks/meso_shared/gaze_exp/derivatives/vdm'
vid_or_name = 'GazeCenterFS_vid.mp4'
new_vid_dir = '/home/ulascombes/disks/meso_shared/gaze_exp/derivatives/vdm'
new_vid_name = 'vdm.mp4'
matrix_name = 'vdm.npy'

# Settings
TR = 1.3
video_size = [140,140]

In [3]:
def matrix_builder(video_path, Tr):
    # Open video
    cap = cv2.VideoCapture(video_path)
    
    # Get video dimensions and frame rate
    frame_width = int(cap.get(3))
    frame_height = int(cap.get(4))
    fps = cap.get(cv2.CAP_PROP_FPS)
    
    # Calculate sampling interval in number of frames
    skip_frames = int(fps * Tr)

    # Initialize numpy array to store sampled frames
    num_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    num_sampled_frames = int(num_frames / skip_frames) 
    frames_np = np.zeros((frame_height, frame_width, num_sampled_frames), dtype=np.uint8)
    
    # Loop over frames in the video
    i = 0
    while(cap.isOpened()):
        ret, frame = cap.read()
        
        if not ret:
            break
        
        # Convert frame to grayscale
        frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        
        # Sample frame
        if i % skip_frames == 0:
            frames_np[:,:,int(i/skip_frames)] = frame_gray
        
        i += 1
        
    # Release resources
    cap.release()
    
    # Convert to binary
    frames_binary = (frames_np > 5).astype(np.uint8)
    
    return frames_binary

In [4]:
#run matrix_builder
vdm_array = matrix_builder('{}/{}'.format(vid_or_dir,vid_or_name), TR)

In [5]:
# make a blank matrix to resize 
height_to_add = int((np.shape(vdm_array)[1]-np.shape(vdm_array)[0])/2)
add = np.zeros([height_to_add,np.shape(vdm_array)[1],np.shape(vdm_array)[2]])

#make concatenation 
add_top = np.concatenate((vdm_array,add), axis=0)
vdm_array_tot = np.concatenate((add,add_top), axis=0)

In [12]:
def video_builder(frames_np, Tr,vdm_size_pix, video_path):
    
    # Downsampling: resize the frames
    frames_reshape = np.zeros((vdm_size_pix[0],vdm_size_pix[1],np.shape(vdm_array)[2]))
    for k in range(frames_reshape.shape[-1]):
        frames_reshape[:,:,k] = cv2.resize(frames_np[:,:,k], dsize=(vdm_size_pix[0], vdm_size_pix[1]), interpolation=cv2.INTER_NEAREST)
    frames = frames_reshape
    
    # matrix shapes
    frame_height, frame_width, num_frames = frames.shape
    
    # make empty video
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    fps = 1/Tr
    video = cv2.VideoWriter(video_path, fourcc, fps, (frame_width, frame_height))
    
    # fill the video
    for i in range(num_frames):
        frame = np.uint8(frames[:, :, i] * 255)
        frame = cv2.cvtColor(frame, cv2.COLOR_GRAY2BGR)
        video.write(frame)
    
    # Save numpy matrix
    np.save('{}/{}'.format(new_vid_dir,matrix_name),frames)

    # Ressources release    
    video.release()

In [11]:
# run video_builder
video_builder(vdm_array_tot,TR,video_size,'{}/{}'.format(new_vid_dir,new_vid_name))